In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from optdesign import *

# Two bond lengths

In [ ]:
# Normally distributed O-H bond lengths in a water molecule
npt = 1000
dmean = 0.9584
dsigma = 0.3

def plot_points(build_design_matrix):
    fig, ax = plt.subplots(dpi=100)
    for _ in range(1):
        ds = np.random.normal(dmean, dsigma, (npt, 2))
        design_matrix = build_design_matrix(ds)
        irows = opt_maxvol(design_matrix)[:design_matrix.shape[1]]
        ax.plot(ds[:, 0], ds[:, 1], "ko", alpha=0.05, zorder=0.5)
        ax.plot(ds[irows, 0], ds[irows, 1], "ro")
    ax.set_title(f"Distribution of {len(irows)} selected bond lengths")
    ax.set_xlabel("Bond length 1")
    ax.set_ylabel("Bond length 2")
    ax.set_aspect("equal")
    fig.tight_layout()
    plt.show(fig)

### Harmonic potential

In [ ]:
def build_design_matrix(ds):
    x0 = (ds[:, 0] - dmean) / dsigma
    x1 = (ds[:, 1] - dmean) / dsigma
    ones = np.ones(len(ds))
    return np.array([ones, x0, x1, 0.5*x0**2, 0.5*x1**2]).T

plot_points(build_design_matrix)

### Haromic potential with cross term

In [ ]:
def build_design_matrix(ds):
    x0 = (ds[:, 0] - dmean) / dsigma
    x1 = (ds[:, 1] - dmean) / dsigma
    ones = np.ones(len(ds))
    return np.array([ones, x0, x1, 0.5*x0**2, 0.5*x1**2, x0*x1]).T

plot_points(build_design_matrix)

### Basis of uncoupled probabilist's Hermite polynomials

In [ ]:
def build_design_matrix(ds):
    x0 = (ds[:, 0] - dmean) / dsigma
    x1 = (ds[:, 1] - dmean) / dsigma
    maxorder = 4
    polys = [np.polynomial.hermite_e.HermiteE.basis(order) for order in range(maxorder + 1)]
    cols = []
    for order, poly in enumerate(polys):
        cols.append(poly(x0))
        if order > 0:
            cols.append(poly(x1))
    return np.array(cols).T

plot_points(build_design_matrix)

### Basis of coupled probabilist's Hermite polynomials

In [ ]:
def build_design_matrix(ds):
    x0 = (ds[:, 0] - dmean) / dsigma
    x1 = (ds[:, 1] - dmean) / dsigma
    maxorder = 4
    polys = [np.polynomial.hermite_e.HermiteE.basis(order) for order in range(maxorder + 1)]
    cols = []
    for order0, poly0 in enumerate(polys):
        for order1, poly1 in enumerate(polys):
            if order0 + order1 <= maxorder:
                cols.append(poly0(x0) * poly1(x1))
    return np.array(cols).T

plot_points(build_design_matrix)